In [2]:
!pip install -q bitsandbytes accelerator transformers langchain ctransformers aiofiles PyPDF2 faiss-gpu pypdf
!pip install ctransformers[cuda]>=0.2.24
!pip install pypdf sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.llms import CTransformers
from langchain.chains import QAGenerationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os
import json
import time
# import uvicorn
import aiofiles
from PyPDF2 import PdfReader
import csv

In [4]:
config = {'max_new_tokens': 2000, 'temperature': 0.2,"gpu_layers":100,"context_length":2500}
llm = CTransformers(
    model = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file='mistral-7b-instruct-v0.1.Q4_K_S.gguf',
    model_type="mistral",
    config=config
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_S.gguf:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

In [12]:
from langchain.tools import BaseTool
from math import pi
from typing import Union
import os
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import CTransformers
from langchain.agents import initialize_agent

from langchain.llms.huggingface_pipeline import HuggingFacePipeline
class WordCountTool(BaseTool):
        name = "Word Length Counter"
        description = "use this tool when you need to count the number of letters in a word"

        def _run(self, word: str):
            return len(word)

        def _arun(self, radius: Union[int, float]):
            raise NotImplementedError("This tool does not support async")

# tools = [WordCountTool()]

In [14]:

tools = [WordCountTool()]
from langchain.agents import AgentType
# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    handle_parsing_errors=True
)
agent("Using custom tool please find number of letters in the word 'Encyclopedia'?")



> Entering new AgentExecutor chain...
Thought: To count the number of letters in a word, I will use the Word Length Counter tool.
Action:
```
{
  "action": "Word Length Counter",
  "action_input": "Encyclopedia"
}
```

Observation: 12
Thought: I now know the final answer
Final Answer: 12

> Finished chain.


{'input': "Using custom tool please find number of letters in the word 'Encyclopedia'?",
 'output': '12'}

In [61]:
import requests
import json

def format_hospital_info(city, category, entries):
    formatted_text = f"\n{city} - {category}:\n"
    for entry in entries:
        formatted_text += f"  Name: {entry['Name']}\n"
        formatted_text += f"  Wait Time: {entry['WaitTime']}\n"
        formatted_text += f"  URL: {entry['URL']}\n"
        formatted_text += f"  Note: {entry['Note']}\n"
        formatted_text += "\n"
    return formatted_text

def Waitime_API_Scrape(word):
    response = requests.get("https://www.albertahealthservices.ca/Webapps/WaitTimes/api/waittimes")

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        text_output = ""

        for city, categories in data.items():
            for category, entries in categories.items():
                text_output += format_hospital_info(city, category, entries)
                # text_output+=entries[1]
        # Print or save the formatted text
        # filepath="waitime.txt"
        # output_file_path = "scrape/"+filepath
        # with open(output_file_path, "w") as file:
        #     file.write(text_output)
        data = {
      "string": text_output
      }
    return text_output[50:100]

In [62]:
class WaitTimeScrapeTool(BaseTool):
        name = "Waitime Data getter"
        description = "use this tool when the question is about Waitimes"

        def _run(self, word: str):
            return Waitime_API_Scrape(word)

        def _arun(self, radius: Union[int, float]):
            raise NotImplementedError("This tool does not support async")


In [63]:
tools = [WaitTimeScrapeTool()]
from langchain.agents import AgentType
# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    handle_parsing_errors=True
)
# agent.agent.llm_chain.prompt.messages[0].prompt.template
agent("What is the Waittime??")



> Entering new AgentExecutor chain...
Thought: The question is asking about wait times, so we should use the Waitime Data getter tool.
Action:
```
{
  "action": "Waitime Data getter",
  "action_input": ""
}
```

Observation: ospital
  Wait Time: 3 hr 5 min
  URL: https://www
Thought: We got the wait time for a hospital, but we need to know what it is.
Action:
```
{
  "action": "Waitime Data getter",
  "action_input": "hospital"
}
```


Observation: ospital
  Wait Time: 3 hr 5 min
  URL: https://www
Thought: We got the wait time for a hospital, but we need to know what it is.
Action:
```
{
  "action": "Waitime Data getter",
  "action_input": "hospital"
}
```

Final Answer: The wait time for a hospital is 3 hours and 5 minutes.

> Finished chain.


{'input': 'What is the Waittime??',
 'output': 'The wait time for a hospital is 3 hours and 5 minutes.'}

In [ ]:
# from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader
# from langchain.embeddings import HuggingFaceEmbeddings
# def scrape_waitime_vectorstore(filepath):
#     DB_FAISS_PATH="vectorstores/db_faiss"
#     DATA_PATH="scrape/"
#     #Create a DirectoryLoader to load all PDFs from the DATA_PATH. Use the PyPDFLoader to load each PDF.
#     loader=DirectoryLoader(DATA_PATH,glob=filepath,loader_cls=TextLoader)
#     documents=loader.load()
#     #shared overlapping text gives some continuity between chunks and context.
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
#     # text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)

#     texts=text_splitter.split_documents(documents)# all the splitted text is here,text chunks
#     # print(texts)
#     embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
#                                      model_kwargs={'device':'cpu'})#creating the embeddings
#     new_db=FAISS.from_documents(texts,embeddings)#using this embedding model,create all the embedding and store it
#     # db.save_local(DB_FAISS_PATH)
#     old_db = FAISS.load_local(DB_FAISS_PATH, embeddings)
#     Removing_vectorstore_docs(old_db)
#     deleted_old_docs_db = FAISS.load_local(DB_FAISS_PATH, embeddings)

#     deleted_old_docs_db.merge_from(new_db)
#     deleted_old_docs_db.save_local(DB_FAISS_PATH)

In [ ]:
if find_urls(message):
        url=find_urls(message)
        url_type=check_url_type(url[0])
    if "waittime" in message:
        filepath=Waitime_API_Scrape()
        scrape_waitime_vectorstore(filepath)
        chain = cl.user_session.get("chain")
        chain = create_chain_qa()
        cl.user_session.set("chain", chain)
        res = await chain.acall(message, callbacks=[cb])